In [1]:
from src.datasets.Nii_Gz_Dataset_3D import Dataset_NiiGz_3D
from src.models.Model_OctreeNCA_3d_patching2 import OctreeNCA3DPatch2
from src.utils.BaselineConfigs import EXP_OctreeNCA3D
from src.utils.Study import Study
from src.utils.ProjectConfiguration import ProjectConfiguration
from src.utils.BaselineConfigs import EXP_OctreeNCA
from src.datasets.Dataset_BCSS_Seg import Dataset_BCSS_Seg
from src.datasets.Dataset_AGGC import Dataset_AGGC
import octree_vis, torch, os, json, openslide, math
import einops
from src.models.Model_OctreeNCAV2 import OctreeNCAV2
import numpy as np
import torch.nn.functional as F
from src.utils.ProjectConfiguration import ProjectConfiguration as pc
from src.models.Model_OctreeNCA_2d_patching2 import OctreeNCA2DPatch2
import matplotlib.pyplot as plt

torch.set_grad_enabled(False)

/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
model_path = "/local/scratch/clmn1/octree_study_new/Experiments/prostatefAbl_none_10_1.0_16_OctreeNCASegmentation/"


with open(os.path.join(model_path, "config.json")) as f:
    config = json.load(f) 

exp = EXP_OctreeNCA3D().createExperiment(config, detail_config={}, dataset_class=Dataset_NiiGz_3D, dataset_args={})

model: OctreeNCA3DPatch2 = exp.model
assert isinstance(model, OctreeNCA3DPatch2)
model = model.eval().cpu()
model.device = "cpu"
for backbone in model.backbone_ncas:
    backbone.device = "cpu"

Loaded configuration is missing keys: '['experiment.run_hash']'. Check if you are loading the correct experiment.
Reload State 2000

-------- Experiment Setup --------
{
    "experiment.name": "prostatefAbl_none_10_1.0_16",
    "experiment.description": "OctreeNCASegmentation",
    "model.output_channels": 1,
    "model.channel_n": 16,
    "model.fire_rate": 0.5,
    "model.kernel_size": [
        3,
        3,
        3,
        3,
        3
    ],
    "model.hidden_size": 64,
    "model.batchnorm_track_running_stats": false,
    "model.train.patch_sizes": [
        [
            80,
            80,
            6
        ],
        [
            80,
            80,
            6
        ],
        null,
        null,
        null
    ],
    "model.train.loss_weighted_patching": false,
    "model.eval.patch_wise": false,
    "model.octree.res_and_steps": [
        [
            [
                320,
                320,
                24
            ],
            10
        ],
     

In [4]:
dataset_test = exp.datasets['test']

In [16]:
sample = dataset_test[0]['image']
sample = torch.from_numpy(sample).float()
sample = einops.rearrange(sample, "h w d -> 1 h w d 1")

In [21]:
model(sample);